In [100]:
with open('/Users/solopov97/Downloads/prorus_text.txt','r') as ru:
    ru=list(ru)
    ru=[e.replace('\n','') for e in ru]
with open('/Users/solopov97/Downloads/prouk_text.txt','r') as uk:
    uk=list(uk)
    uk=[e.replace('\n','') for e in uk]

In [89]:
with open('/Users/solopov97/Downloads/telegram_fake.txt','r') as fru:
    fru=list(fru)
    fru=[e.replace('\n','') for e in fru]
with open('/Users/solopov97/Downloads/telegram_real.txt','r') as fuk:
    fuk=list(fuk)
    fuk=[e.replace('\n','') for e in fuk]

In [54]:
with open('/Users/solopov97/Downloads/french_fake.txt','r') as fru:
    fru=list(fru)
    fru=[e.replace('\n','') for e in fru]
with open('/Users/solopov97/Downloads/french_real.txt','r') as fuk:
    fuk=list(fuk)
    fuk=[e.replace('\n','') for e in fuk]

In [111]:
with open('/Users/solopov97/Downloads/prorus_text_nofrench.txt','r') as ru:
    ru=list(ru)
    ru=[e.replace('\n','') for e in ru]
with open('/Users/solopov97/Downloads/prouk_text_nofrench.txt','r') as uk:
    uk=list(uk)
    uk=[e.replace('\n','') for e in uk]

In [90]:
labels_fuk=[1 for e in fuk]
labels_fru=[0 for e in fru]

In [76]:
len(fuk), len(fru)

(7263, 61595)

In [112]:
labels_uk=[1 for e in uk]
labels_ru=[0 for e in ru]

In [79]:
u = {'X':fuk,'Y':labels_fuk}
df_fuk = pd.DataFrame(u)
r = {'X':fru,'Y':labels_fru}
df_fru = pd.DataFrame(r)
dff=pd.concat([df_fuk,df_fru], axis=0)
Xf=dff['X'].values.tolist()
Yf=dff['Y'].values.tolist()

In [113]:
u = {'X':uk,'Y':labels_uk}
df_uk = pd.DataFrame(u)
r = {'X':ru,'Y':labels_ru}
df_ru = pd.DataFrame(r)
df=pd.concat([df_uk,df_ru], axis=0)
Xf=df['X'].values.tolist()
Yf=df['Y'].values.tolist()

In [93]:
dfall=pd.concat([df_uk,df_ru,df_fuk,df_fru], axis=0)

In [114]:
train_df, test_df = train_test_split(df, test_size=0.10, random_state=80)
X=test_df['X'].values.tolist()
Y=test_df['Y'].values.tolist()
print(len(Y))

1768


In [107]:
model = ClassificationModel(
    "bert", "/Users/solopov97/Desktop/GERMANY/DR.SOLOPOVA/features/out_propaganda/1/outputs/best_model",use_cuda=False)
predictions1, raw_outputs1 = model.predict(X)

  0%|          | 0/1827 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/229 [00:00<?, ?it/s]

In [72]:
print(predictions)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [99]:
import sklearn
Y=test_df['Y'].values.tolist()
from sklearn.metrics import cohen_kappa_score
k=sklearn.metrics.cohen_kappa_score(predictions, Y)
k

0.8180761020249265

In [109]:
k=sklearn.metrics.cohen_kappa_score(predictions1,Y)
print(k)
from sklearn.metrics import f1_score
f1=f1_score(Y, predictions1 ,
            average='micro')
print(f1)

0.864886932294056
0.9326765188834154


In [110]:
fp=0
fn=0
tp=0
tn=0
for tr,pr in zip(Y, predictions1):
    if tr==pr and tr==1:
        tp+=1
    elif tr==pr and tr==0:
        tn+=1
    elif tr!=pr and tr==1:
        fn+=1
    elif tr!=pr and tr==0:
        fp+=1   
print(fp,fn,tp,tn)


12 111 779 925


In [94]:
counter=2
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
seeds=[24,42,10,80,63]
seeds=[80]
all_outputs=[]
all_results=[]
true=[]
pred=[]
def f1_multiclass(labels, preds):
    true.append(labels)
    pred.append(preds)
    return f1_score(labels, preds, average='micro')

In [95]:
for e in seeds:
    train_args ={"reprocess_input_data": True,
              "fp16":False,
              "num_train_epochs": 4,
            "save_model_every_epoch": True,
            "output_dir":'/Users/solopov97/Desktop/GERMANY/DR.SOLOPOVA/features/out_propaganda/'+str(counter)+'/outputs'}
    train_df, test_df = train_test_split(dfall, test_size=0.10, random_state=e)
    model = ClassificationModel("bert","bert-base-multilingual-cased",num_labels=2,
                            args=train_args,use_cuda=False)
    print('model ready')
    model.train_model(train_df,overwrite_output_dir=True)
    print('train ready')
    result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)
    all_results.append(result)
    all_outputs.append(model_outputs)
    print(result)
    counter+=1

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

model ready


/Users/solopov97/opt/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:585: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/29506 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

/Users/solopov97/opt/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1426: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3279 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation:   0%|          | 0/410 [00:00<?, ?it/s]

{'mcc': 0.8181791737286435, 'tp': 1440, 'tn': 1541, 'fp': 136, 'fn': 162, 'auroc': 0.9649387281997682, 'auprc': 0.9616197730852462, 'f1': 0.9091186337297956, 'acc': 0.9091186337297956, 'eval_loss': 0.40182820134796204}


In [96]:
predictions, raw_outputs = model.predict(test_df['X'].values.tolist())

  0%|          | 0/3279 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tok

  0%|          | 0/410 [00:00<?, ?it/s]

In [68]:
print(len(predictions))

40


In [18]:
f1s=[]
for i in all_results:
    print(i['f1'])
    f1s.append(i['f1'])

0.9117082533589251
0.9232245681381958
0.9238643634037108
0.9289827255278311
0.9174664107485605


In [19]:
sum(f1s)/len(f1s)

0.9210492642354445